In [3]:
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

# -------------------------------
# 读取数据
file_path = 'rawdata/数据集B.xlsx'
data = pd.read_excel(file_path)

# 更新基线特征列表和目标变量名称（添加了PANSS相关变量）
baseline_features = [
 'rm_1750_acc at baseline', 'rm_750_acc at baseline', 'stroop_incongruent_rt  at baseline',
 'stroop_interference_effect_rt  at baseline', 'nogo_acc  at baseline', 'switch_cost  at baseline',
 'dsbt_span at baseline', 'Age', 'Education years', 'BMI', 'SES', 'Gender', 'Ethnic', 'Residence',
 'Only child', 'Smoking status', 'Alcohol consumption', 'Employed', 'Marital status', 'First episode',
 'Frequency of episodes', 'Age at onset', 'Duration of disorder', 'Dose equivalent to olanzapine (mg/day)'
]
target = 'prob_change'

# 选取需要处理的列：基线特征与目标变量
columns_to_use = baseline_features + [target]
df = data[columns_to_use]

# 筛选数值型变量（包括目标变量）
numeric_cols = [col for col in df.columns if pd.api.types.is_numeric_dtype(df[col])]
# 如果不希望修改目标变量，则将其从数值型变量列表中排除
numeric_cols_features = [col for col in numeric_cols if col != target]

# -----------------------------------------
# 1. 统计缺失值
print("【原始数据缺失值统计】")
missing_counts = df.isnull().sum()
print(missing_counts)
print()

# -----------------------------------------
# 2. 统计异常值（基于 IQR 方法）——仅针对特征变量，不包括目标变量
def count_outliers_iqr(df, columns, factor=1.5):
    """
    统计每个变量中低于 (Q1 - factor*IQR) 或高于 (Q3 + factor*IQR) 的异常值数量。
    """
    outlier_counts = {}
    for col in columns:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - factor * IQR
        upper_bound = Q3 + factor * IQR
        count = ((df[col] < lower_bound) | (df[col] > upper_bound)).sum()
        outlier_counts[col] = count
    return outlier_counts

print("【原始数据异常值统计】")
outlier_counts = count_outliers_iqr(df, numeric_cols_features, factor=1.5)
for col, count in outlier_counts.items():
    print(f"{col}: {count} 个异常值")
print()

# -----------------------------------------
# 3. 对异常值进行填充处理（winsorization：将超出上下界的值替换为上下界）——仅针对特征变量，不包括目标变量
def cap_outliers_iqr(df, columns, factor=1.5):
    """
    对每个数值型变量，计算下界和上界（下界 = Q1 - factor*IQR, 上界 = Q3 + factor*IQR），
    然后将超出此范围的值替换为相应的边界值（不删除数据）。
    """
    df_capped = df.copy()
    for col in columns:
        Q1 = df_capped[col].quantile(0.25)
        Q3 = df_capped[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - factor * IQR
        upper_bound = Q3 + factor * IQR
        print(f"{col}: 下界 = {lower_bound:.2f}, 上界 = {upper_bound:.2f}")
        # 使用clip函数将超出范围的值替换为上下界
        df_capped[col] = df_capped[col].clip(lower=lower_bound, upper=upper_bound)
    return df_capped

# 对所有特征变量进行异常值填充（截断处理），目标变量不参与处理
df_features_capped = cap_outliers_iqr(df, numeric_cols_features, factor=1.5)

# 将目标变量原样保留
df_capped = df_features_capped.copy()
df_capped[target] = df[target]

print()

# 检查异常值填充后是否还有缺失值，并用中位数填充缺失值（如果有）
print("【异常值填充后缺失值统计】")
print(df_capped.isnull().sum())
print()

# 进行缺失值填充（本例中原始数据没有缺失值，但此步骤可以保证数据完整性）
df_filled = df_capped.fillna(df_capped.median())

# -----------------------------------------
# 4. 标准化处理：仅对连续变量进行标准化，分类变量保持不变
# 定义不需要标准化的分类变量列表
categorical_vars = ['Gender', 'Ethnic', 'Residence', 'Only child', 'Smoking status', 
                    'Alcohol consumption', 'Employed', 'Marital status', 'First episode']

# 连续变量：从baseline_features中排除分类变量
continuous_features = [col for col in baseline_features if col not in categorical_vars]

print("【待标准化的连续变量】")
print(continuous_features)
print()

scaler = StandardScaler()
df_filled[continuous_features] = scaler.fit_transform(df_filled[continuous_features])

# -----------------------------------------
# 对比预处理前后的数据行数（数据行数应保持不变）
print("原始数据行数:", df.shape[0])
print("预处理后数据行数:", df_filled.shape[0])
print()

# 将预处理后的数据保存到 Excel
output_path = "./preprocessed_data_filled.xlsx"
df_filled.to_excel(output_path, index=False)
print(f"预处理后的数据已保存至 {output_path}")

【原始数据缺失值统计】
rm_1750_acc at baseline                       0
rm_750_acc at baseline                        0
stroop_incongruent_rt  at baseline            0
stroop_interference_effect_rt  at baseline    0
nogo_acc  at baseline                         0
switch_cost  at baseline                      0
dsbt_span at baseline                         0
Age                                           0
Education years                               1
BMI                                           0
SES                                           0
Gender                                        0
Ethnic                                        0
Residence                                     0
Only child                                    0
Smoking status                                0
Alcohol consumption                           0
Employed                                      0
Marital status                                0
First episode                                 0
Frequency of episodes       